In [ ]:
# Abdülhalim YAYDEMİR

### Eğitim verilerinden priors ve likelihoods bulma

In [4]:
# Kütüphaneler 

import pandas as pd
import numpy as np

# verileri eğitim ve test olarak ayırmak için kullanıldı.
# metinleri vektör yapmak için kullanıldı.
from sklearn.model_selection import train_test_split  
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 

# En son karşılaştırma amaçlı kullanıldı.
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


# Dosya okuma ve düzenleme
df = pd.read_csv('spam.csv',encoding='ISO-8859-1')
veri = df.copy()
veri = veri.drop(["Unnamed: 0"], axis=1)
veri = veri.rename(columns={"label":"etiket","text":"metin","label_num":"etiket_num"})


# Metin içeriği düzenleme (küçük harfe çevirme)
tumcumleler = []
for i in range(len(veri["metin"].values)):
    r1 = veri["metin"].values[i]    
    temizcumle = []
    cumleler = r1.lower()
    
    for kelimeler in cumleler.split():
        temizcumle.append(kelimeler)
        
    tumcumleler.append(" ".join(temizcumle))

veri["yeni_metin"] = tumcumleler



# vektör oluşturma
cv = CountVectorizer()
cv.fit(veri["yeni_metin"])
x = cv.fit_transform(veri["yeni_metin"]).toarray()
y = veri["etiket_num"]



# eğitim ve test verileri olarak ayırma
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33)



# Eğitim verilerinden Prior'lar hesaplama 
spam_count = 0.0
for i in range(np.shape(y_train)[0]):
    if y_train.values[i]==1:
        spam_count=spam_count+1.0
        
ham_count=(np.shape(y_train)[0])-spam_count

P_spam = spam_count/(np.shape(y_train)[0])
P_ham = ham_count/(np.shape(y_train)[0])

print("Priors:")
print("P_spam = ",P_spam)
print("P_ham = ",P_ham)




# Eğitim verilerinden Likelihood'lar hesaplandı
P_x_spam = np.arange(np.shape(x_train)[1])
P_x_ham = np.arange(np.shape(x_train)[1])
for j in range(np.shape(x_train)[1]):
    x_spam_count = 0.0
    x_ham_count = 0.0
    for i in range(np.shape(x_train)[0]):
        if (x_train[i][j]!=0) & (y_train.values[i]==1):
            x_spam_count = x_spam_count + 1.0
        if (x_train[i][j]!=0) & (y_train.values[i]==0):
            x_ham_count = x_ham_count + 1.0
    P_x_spam[j] = x_spam_count
    P_x_ham[j] = x_ham_count
    
P_x_spam = P_x_spam/spam_count
P_x_ham = P_x_ham/ham_count 

print("Likelihoods: ")
print("P_x_spam = ",P_x_spam)
print("P_x_ham = ",P_x_ham)

Priors:
P_spam =  0.28983833718244806
P_ham =  0.710161662817552
Likelihoods: 
P_x_spam =  [0.08665339 0.07669323 0.00298805 ... 0.00099602 0.00099602 0.00099602]
P_x_ham =  [0.17113821 0.20772358 0.00081301 ... 0.         0.         0.        ]


### Test metinlerinin etiketlerini tahmin etme, doğruluk hesabı

In [5]:
# Test metin verileri için etiket verileri tahmin edildi.
y_predict = np.arange(np.shape(x_test)[0])
for i in range(np.shape(x_test)[0]):
    NaiveBayes_spam = 1.0
    NaiveBayes_ham = 1.0
    for j in range(np.shape(x_test)[1]):
        if (x_test[i][j]!=0):
            NaiveBayes_spam = NaiveBayes_spam * P_x_spam[j]
            NaiveBayes_ham = NaiveBayes_ham * P_x_ham[j]
            
    NaiveBayes_spam = NaiveBayes_spam * P_spam
    NaiveBayes_ham = NaiveBayes_ham * P_ham
    if (NaiveBayes_spam>=NaiveBayes_ham):
        y_predict[i] = 1
    if (NaiveBayes_spam<NaiveBayes_ham):
        y_predict[i] = 0
    
    
    
# Tahmin etiket verileri ile test etiket verileri kıyaslandı
predict_correct = 0.0
predict_wrong = 0.0
for i in range(np.shape(x_test)[0]):
    if (y_predict[i] == y_test.values[i]):
        predict_correct = predict_correct + 1.0
    if (y_predict[i] != y_test.values[i]):
        predict_wrong = predict_wrong + 1.0
        
        
        
# Doğruluk hesabı yapıldı
accuracy = predict_correct/(predict_correct+predict_wrong)
print("Accuracy = ",accuracy)



Accuracy =  0.6438195664909198


### Sklearn ile doğruluk hesabı 

In [6]:
# Sklearn kütüphanesi ile karşılaştırma yapıldı
model=MultinomialNB()
model.fit(x_train,y_train)
tahmin=model.predict(x_test)
accuracy_NB=accuracy_score(y_test,tahmin)
print("Accuracy_NB = ",accuracy_NB)

Accuracy_NB =  0.9818394844756884


#### Sonuç 
#### Naive Bayes manuel olarak uygulandığında % 64 doğruluk bulunmuşken, aynı veri Sklearn tarafından % 98 bulunmuştur.
#### Bu değer veri setinin ön işlem sürecini iyileştirerek (noktalama işaretleri,sayılar, tekrar sayısı, gereksiz sözçükler) 
#### ve farklı bir vektör oluşturma yöntemi ile iyileştirilebilir.